In [1]:
import numpy as np
import pandas as pd

from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV

import math

from readers_preprocess import read_filter
from time_to_freq_domain import eval_psd, eval_psd_not_modulated, fft_signal, eval_band_power

from dim_reduction import  reduce_features, pca_data, select_best_chanels
from transformers import flatten_data, transform_to_one_chanel_data
from time_domain_features import make_fets

# Criterion for evaluation(Accuracy of KNN)

In [3]:
def eval_feats(X_train, X_test, y_train, y_test, clf):
    """
    Fit a classifier and evaluate its accuracy on a test set
    """
    clf.fit(X_train, y_train) 
    y_pred = clf.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    print(acc)
    return acc


def eval_feats_with_KNN(train_data_lst, test_data_lst, y_train, y_test, data_anots):
    """
    Evaluate subset of features with KNN based on accuracy on a test set
    """
    acc_list = []
    new_data_anots = []
    for feats_idx in range(len(train_data_lst)):
        clf = KNeighborsClassifier(n_neighbors=3)
        X_train = train_data_lst[feats_idx]
        X_test = test_data_lst[feats_idx]
        acc = eval_feats(X_train, X_test, y_train, y_test, clf)
        acc_list.append(acc)
        new_data_anots.append(data_anots[feats_idx]+"_KNN")
    return acc_list, new_data_anots


def eval_feats_tune_KNN(train_data_lst, test_data_lst, y_train, y_test, data_anots):
    """
    Make huperparamter optimization of knn, next evalaute a subset of features using 
    knn and accuracy metric
    """
    parameter_candidates = [{'n_neighbors': [3, 4, 5, 6, 7]}]
    #pdb.set_trace()
    acc_list = []
    new_data_anots = []
    for feats_idx in range(len(train_data_lst)):
        clf = GridSearchCV(estimator=KNeighborsClassifier(), cv=5 , param_grid=parameter_candidates)

        X_train = train_data_lst[feats_idx]
        X_test = test_data_lst[feats_idx]
        acc = eval_feats(X_train, X_test, y_train, y_test, clf)
        acc_list.append(acc)
        new_data_anots.append(data_anots[feats_idx]+"_KNN")
    return acc_list, new_data_anots

In [4]:

all_paths = [['data_bci\\row_data\\subject1\\'], ['data_bci\\row_data\\subject2\\'], ['data_bci\\row_data\\subject3\\']]
columns_to_read =  ['Fp1', 'AF3' ,'F7', 'F3', 'FC1', 'FC5', 'T7', 'C3', 'CP1', 'CP5',
                   'P7', 'P3', 'Pz', 'PO3', 'O1', 'Oz', 'O2', 'PO4', 'P4', 'P8', 'CP6',
                   'CP2', 'C4', 'T8', 'FC6', 'FC2', 'F4', 'F8', 'AF4', 'Fp2', 'Fz', 'Cz','class']

test_subject = '02'
freq = 512
train_subjects = ['01']

min_freq = 8
subject_results = []
knn_results = []
#chanels_acc, chanels_rank = select_best_chanels()

chanels_rank =  [25,  4,  9, 26, 30, 11,  7]
#Nyquist criterion determination of freq resolution
window_size = 2*freq/min_freq
window_size

128.0

# Pipeline Selection

Which of the following preprcessing steps yields better results: 
 1. Train signal lenght:<br>
     1.1. Useing the full lenghts of the signal<br>
     1.2. Useing 1 sec segments<br>    
 2. Spectral Filtering<br>
     2.1. Bandpass filter<br>
     2.2. Without bandpass filter<br>    
 3. Choice of window function<br>
     3.1 Boxcar<br>
     3.2 Hanning<br>
     3.3 Ad hoc calclated signal properties in the time domain
 4. Spectral features<br>
     4.1. Energy at all waves 8-45 hz with 4 hz spectral resolution<br>
     4.2. Energy in certain bandas(8-12 hz, 16-32 hz, 36-44 hz)<br>
 5. Feature selection:<br>
     5.1 Removal of features that don't correlate with the targaet variable<br>
     and features that correlate between eachother<br>
     5.2 Don't apply features selection<br>
 6. Dimensionality reduction:<br>
     6.1. PCA<br>
     6.2. Don't apply PCA<br>

Applying transofrmation 2-6 to full lenghts segments and evaluauting on one sec. segments

# Full lenght  signals - Welch transformation

In [5]:
def create_feature_represnetations(time_domain_signals, num_perseg=128, num_overlap=64, freq=512, min_freq=8, max_freq=45):
    """
    Transforms  signals from A(time) to Energy(frequencies) 
    The transforation is done via the welch methood with boxcar/hanning windows
    also make one more features representation energy in a given frequencie band
    """
    psd_cut1 = 2
    psd_cut2 = -3
    psd_freq_res = 4
    
    data_modulated_freq_domain = [eval_psd(data, num_perseg, num_overlap, freq, min_freq, max_freq) for data in time_domain_signals]
    data_modulated_freq_band = [eval_band_power(data, psd_freq_res, psd_cut1, psd_cut2) for data in data_modulated_freq_domain]
    
    data_freq_domain = [eval_psd_not_modulated(data, num_perseg, num_overlap, freq, min_freq, max_freq) for data in time_domain_signals]
    data_freq_band = [eval_band_power(data, psd_freq_res, psd_cut1, psd_cut2) for data in data_freq_domain]

    
    data_representations = data_modulated_freq_domain+data_modulated_freq_band+data_freq_domain+data_freq_band
    data_anots = [
        'hanning', 'filtered_hanning', 'hanning_bandpower', 'filtered_hanning_bandpower',
        'boxcar', 'filtered_boxcar', 'boxcar_bandpower', 'filtered_boxcar_bandpower',
        ]
    
    return data_representations, data_anots

In [6]:

welch_knn_results = []
for path in all_paths:
    cutoff_beggining = 0
    seq_len = 0
    cut_step = 0
    train_data, train_data_filtered, train_anots, test_data, test_data_filtered, test_annoations = read_filter(path, train_subjects, test_subject, columns_to_read, cutoff_beggining, seq_len, cut_step)
    seq_len = freq
    cut_step = int(seq_len/2)
    window_train_data, window_train_data_filtered, window_train_anots, window_test_data, window_test_data_filtered, window_test_annoations = read_filter(path, train_subjects, test_subject, columns_to_read, cutoff_beggining, seq_len, cut_step)
    
    
    
    train_time_domain_signals = [train_data, train_data_filtered]    
    test_time_domain_signals = [window_test_data, window_test_data_filtered]

    train_freq_domain_signals, data_anots = create_feature_represnetations(train_time_domain_signals)
    test_freq_domain_signals, _ = create_feature_represnetations(test_time_domain_signals)
    train_data_flatten = [flatten_data(data[:,:,chanels_rank]) for data in train_freq_domain_signals]
    test_data_flatten = [flatten_data(data[:,:,chanels_rank]) for data in test_freq_domain_signals]
    
    train_data_reduced, test_data_reduced, reduced_data_anots = reduce_features(train_data_flatten, test_data_flatten, train_anots, data_anots)
    train_data_pca, test_data_pca, pca_data_anots = pca_data(train_data_flatten+train_data_reduced, test_data_flatten+test_data_reduced, data_anots+reduced_data_anots)
    
    train_lst = train_data_flatten + train_data_reduced + train_data_pca
    test_lst = test_data_flatten + test_data_reduced + test_data_pca
    data_representations = data_anots + reduced_data_anots + pca_data_anots
    acc_list, welch_data_anots = eval_feats_with_KNN(train_lst, test_lst, train_anots, window_test_annoations, data_representations)
    welch_knn_results.append(acc_list)

0.574889867841
0.387665198238
0.579295154185
0.387665198238
0.621145374449
0.387665198238
0.438325991189
0.387665198238
0.605726872247
0.387665198238
0.623348017621
0.387665198238
0.605726872247
0.387665198238
0.599118942731
0.387665198238
0.581497797357
0.387665198238
0.581497797357
0.387665198238
0.616740088106
0.387665198238
0.440528634361
0.387665198238
0.607929515419
0.387665198238
0.612334801762
0.387665198238
0.614537444934
0.387665198238
0.588105726872
0.387665198238
0.407079646018
0.340707964602
0.41592920354
0.340707964602
0.41592920354
0.340707964602
0.438053097345
0.340707964602
0.402654867257
0.340707964602
0.404867256637
0.340707964602
0.422566371681
0.340707964602
0.426991150442
0.340707964602
0.409292035398
0.340707964602
0.420353982301
0.340707964602
0.413716814159
0.340707964602
0.429203539823
0.340707964602
0.411504424779
0.340707964602
0.402654867257
0.340707964602
0.422566371681
0.340707964602
0.420353982301
0.340707964602
0.466666666667
0.342222222222
0.4711111111

Report the results

In [7]:
results = [[],[],[]]
results[0] = welch_knn_results[0]
results[1] = welch_knn_results[1]
results[2] = welch_knn_results[2]
result_anots  = data_representations

# Create report 
df = pd.DataFrame(results, columns=result_anots).T
df.columns = ['person_1_accuracy', 'person_2_accuracy', 'person_3_accuracy']
df.index.name = 'pipeline'
m = df.mean(axis=1)
s = df.std(axis=1)
df['mean_accuracy'] = m
df['std_accuracy'] = s
df.sort_values(by='mean_accuracy', ascending=False, inplace=True)
df = df.round(3)
df.to_excel("Pipeline_selection_report_15sec.xlsx")
df

,person_1_accuracy,person_2_accuracy,person_3_accuracy,mean_accuracy,std_accuracy
pipeline,,,,,
boxcar_reduced_pca,0.615,0.423,0.518,0.518,0.096
boxcar_reduced,0.606,0.423,0.520,0.516,0.092
hanning_reduced_pca,0.608,0.412,0.524,0.515,0.099
hanning_reduced,0.606,0.403,0.529,0.512,0.103
boxcar_pca,0.617,0.414,0.500,0.510,0.102
boxcar,0.621,0.416,0.493,0.510,0.104
hanning_bandpower_reduced,0.623,0.405,0.484,0.504,0.111
boxcar_bandpower_reduced,0.599,0.427,0.467,0.498,0.090
boxcar_bandpower_reduced_pca,0.588,0.420,0.467,0.492,0.087


In [53]:
['hanning','boxcar', 'filtered', 'bandpower', 'reduced','pca']
data = df.reset_index()
eval_function  = [np.mean, np.max]
pca_res = data[data['pipeline'].apply(lambda x: 'pca' in x)]['mean_accuracy'].apply(eval_function)
reduced_res = data[data['pipeline'].apply(lambda x: 'reduced' in x)]['mean_accuracy'].apply(eval_function)
filtered_res = data[data['pipeline'].apply(lambda x: 'filtered' in x)]['mean_accuracy'].apply(eval_function)
bandpower_res = data[data['pipeline'].apply(lambda x: 'bandpower' in x)]['mean_accuracy'].apply(eval_function)
hanning_res = data[data['pipeline'].apply(lambda x: 'hanning' in x)]['mean_accuracy'].apply(eval_function)
boxcar_res = data[data['pipeline'].apply(lambda x: 'boxcar' in x)]['mean_accuracy'].apply(eval_function)

column_names = ['pca', 'reduced', 'hanning', 'boxcar', 'bandpower','filtered']
table = pd.concat([pca_res, reduced_res, hanning_res, boxcar_res, bandpower_res, filtered_res], axis=1)
table.columns = column_names
table.round(3)

,pca,reduced,hanning,boxcar,bandpower,filtered
mean,0.423,0.431,0.427,0.421,0.415,0.357
amax,0.518,0.518,0.515,0.518,0.504,0.357


Applying transofrmation 2-6 to 1 sec. segments and evaluauting on one sec. segments

# One second signals- Time domain and Freq. Domain

In [16]:
def create_AD_hoc_feature_represnetations(time_domain_data, norm_consts, freq):
    """
    Calcluate signals propperties in the time domain
    """
    
    feats = make_fets(time_domain_data[0]/norm_consts[0], freq)
    filterd_feats = make_fets(time_domain_data[1]/norm_consts[1], freq)
    
    return [feats, filterd_feats],  ['stat.properies', 'filtered_stat.properies']
     
    
def normalize(data, max_elements, min_elements):
    """
    MinMax normalize the data
    """
    max_el = max_elements[0]
    min_el = min_elements[0]
    feats = (data[0]-min_el)/(max_el-min_el)
    
    max_el = max_elements[1]
    min_el = min_elements[1]
    filterd_feats = (data[1]-min_el)/(max_el-min_el)   
    filterd_feats = np.nan_to_num(filterd_feats)

    return [feats, filterd_feats]


one_sec_knn_results = []
for path in all_paths:
    seq_len = freq
    cut_step = int(seq_len/2)
    cutoff_beggining = 0
    
    window_train_data, window_train_data_filtered, window_train_anots, window_test_data, window_test_data_filtered, window_test_annoations = read_filter(path, train_subjects, test_subject, columns_to_read, cutoff_beggining, seq_len, cut_step)

    
    train_time_domain_signals = [window_train_data, window_train_data_filtered]   
    test_time_domain_signals = [window_test_data, window_test_data_filtered]
    
    train_freq_domain_signals, data_anots = create_feature_represnetations(train_time_domain_signals)
    test_freq_domain_signals, _ = create_feature_represnetations(test_time_domain_signals)

    norm_consts = [np.abs(data).mean() for data in train_time_domain_signals]
    train_time_domain_feats, time_domain_anots  =  create_AD_hoc_feature_represnetations(train_time_domain_signals, norm_consts, freq)
    test_time_domain_feats, _  =  create_AD_hoc_feature_represnetations(test_time_domain_signals, norm_consts, freq)
    data_anots += time_domain_anots


    
    

    train_freq_domain_feats = [flatten_data(data[:,:,chanels_rank]) for data in train_freq_domain_signals]
    test_freq_domain_feats = [flatten_data(data[:,:,chanels_rank]) for data in test_freq_domain_signals]
    
    train_time_domain_feats = [flatten_data(data[:,:,chanels_rank]) for data in train_time_domain_feats]
    test_time_domain_feats = [flatten_data(data[:,:,chanels_rank]) for data in test_time_domain_feats]
    max_elements = [data.max(axis=0) for data in train_time_domain_feats]
    min_elements = [data.min(axis=0) for data in train_time_domain_feats]
    train_time_domain_feats = normalize(train_time_domain_feats, max_elements, min_elements)
    test_time_domain_feats = normalize(train_time_domain_feats, max_elements, min_elements) 

    train_data_flatten = train_freq_domain_feats+train_time_domain_feats
    test_data_flatten = test_freq_domain_feats+test_time_domain_feats
    

    train_data_reduced, test_data_reduced, reduced_data_anots = reduce_features(train_data_flatten, test_data_flatten, window_train_anots, data_anots)
    train_data_pca, test_data_pca, pca_data_anots = pca_data(train_data_flatten+train_data_reduced, test_data_flatten+test_data_reduced, data_anots+reduced_data_anots)
    
    train_lst = train_data_flatten + train_data_reduced + train_data_pca
    test_lst = test_data_flatten + test_data_reduced + test_data_pca
    data_representations = data_anots + reduced_data_anots + pca_data_anots

    acc_list, one_sec_data_anots = eval_feats_tune_KNN(train_lst, test_lst, window_train_anots, window_test_annoations, data_representations)
    one_sec_knn_results.append(acc_list)

2
2


C:\Users\EMO\Anaconda3\lib\site-packages\ipykernel_launcher.py:42: RuntimeWarning: invalid value encountered in true_divide
C:\Users\EMO\Anaconda3\lib\site-packages\ipykernel_launcher.py:42: RuntimeWarning: divide by zero encountered in true_divide
C:\Users\EMO\Anaconda3\lib\site-packages\numpy\lib\function_base.py:2530: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
C:\Users\EMO\Anaconda3\lib\site-packages\numpy\lib\function_base.py:2531: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
C:\Users\EMO\Anaconda3\lib\site-packages\numpy\core\_methods.py:36: RuntimeWarning: overflow encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims, initial)


train
0.526431718062
0.314977973568
0.517621145374
0.31718061674
0.480176211454
0.299559471366
0.387665198238
0.299559471366
0.416299559471


C:\Users\EMO\Anaconda3\lib\site-packages\numpy\core\_methods.py:36: RuntimeWarning: overflow encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims, initial)


0.361233480176
0.533039647577
0.301762114537
0.526431718062
0.31718061674
0.460352422907
0.301762114537
0.398678414097
0.306167400881
0.359030837004


...

train
0.435555555556
0.324444444444
0.411111111111
0.324444444444
0.408888888889
0.324444444444
0.431111111111
0.324444444444
0.353333333333


C:\Users\EMO\Anaconda3\lib\site-packages\numpy\core\_methods.py:36: RuntimeWarning: overflow encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims, initial)


0.324444444444
0.433333333333
0.324444444444
0.408888888889
0.324444444444
0.382222222222
0.324444444444
0.42
0.324444444444
0.333333333333


C:\Users\EMO\Anaconda3\lib\site-packages\numpy\core\_methods.py:36: RuntimeWarning: overflow encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims, initial)


0.324444444444
0.431111111111
0.324444444444
0.426666666667
0.324444444444
0.402222222222
0.324444444444
0.424444444444
0.324444444444
0.368888888889
0.324444444444
0.448888888889
0.324444444444
0.402222222222
0.324444444444
0.417777777778
0.324444444444
0.431111111111
0.324444444444
0.333333333333
0.337777777778


Report the results

In [24]:
results = [[],[],[]]
results[0] = one_sec_knn_results[0]
results[1] = one_sec_knn_results[1]
results[2] = one_sec_knn_results[2]
result_anots  = one_sec_data_anots

# Create report 
df = pd.DataFrame(results, columns=result_anots).T
df.columns = ['person_1_accuracy', 'person_2_accuracy', 'person_3_accuracy']
df.index.name = 'pipeline'
m = df.mean(axis=1)
s = df.std(axis=1)
df['mean_accuracy'] = m
df['std_accuracy'] = s
df.sort_values(by='mean_accuracy', ascending=False, inplace=True)
df = df.round(3)
freq_domain_rows = df.reset_index()['pipeline'].apply( lambda x: 'stat.properies' not in x)
freq_domain_report = df[freq_domain_rows.values]
time_domain_report = df[~freq_domain_rows.values]
freq_domain_report.to_excel("Pipeline_selection_freq_1sec.xlsx")
time_domain_report.to_excel("Pipeline_selection_time_1sec.xlsx")
freq_domain_report, time_domain_report

(                                            person_1_accuracy  \
 pipeline                                                        
 hanning_pca_KNN                                         0.555   
 hanning_reduced_pca_KNN                                 0.520   
 hanning_bandpower_pca_KNN                               0.522   
 hanning_reduced_KNN                                     0.533   
 hanning_KNN                                             0.526   
 hanning_bandpower_KNN                                   0.518   
 hanning_bandpower_reduced_KNN                           0.526   
 boxcar_reduced_pca_KNN                                  0.482   
 hanning_bandpower_reduced_pca_KNN                       0.511   
 boxcar_KNN                                              0.480   
 boxcar_pca_KNN                                          0.456   
 boxcar_bandpower_reduced_pca_KNN                        0.403   
 boxcar_reduced_KNN                                      0.460   
 boxcar_ba

# Choose time resoltion(lenght of the window)

In [13]:
from dim_reduction import rank_features
from sklearn.decomposition import PCA
knn_results = []
subject_results = []

num_perseg_lst= [128, 256, 512]
overlap_lst = [int(seg_len/2) for seg_len in num_perseg_lst]
freq=512
min_freq = 8
max_freq = 45
resolutions_accs = [[],[],[]]


for path in all_paths:
    cutoff_beggining = 0
    seq_len = 0
    cut_step = 0
    train_data, _, train_anots, test_data, _, test_annoations = read_filter(path, train_subjects, test_subject, columns_to_read, cutoff_beggining, seq_len, cut_step)
    seq_len = freq
    cut_step = int(seq_len/2)
    window_train_data, _, window_train_anots, window_test_data, _, window_test_annoations = read_filter(path, train_subjects, test_subject, columns_to_read, cutoff_beggining, seq_len, cut_step)
    y_train = train_anots
    y_test = window_test_annoations
    
    for resolution_idx in range(len(num_perseg_lst)):
        num_perseg = num_perseg_lst[resolution_idx]
        num_overlap = overlap_lst[resolution_idx]
        
        
        X_train =  eval_psd_not_modulated(train_data, num_perseg, num_overlap, freq, min_freq, max_freq)
        X_test =  eval_psd_not_modulated(window_test_data, num_perseg, num_overlap, freq, min_freq, max_freq)

        X_train = flatten_data(X_train[:,:,chanels_rank])
        X_test = flatten_data(X_test[:,:,chanels_rank]) 
        
        most_important_features = rank_features(X_train, y_train)
        
        X_train = X_train[:, most_important_features]
        X_test = X_test[:, most_important_features]
        
        pca = PCA(n_components=0.985)
        X_train = pca.fit_transform(X_train)
        X_test = pca.transform(X_test)
        clf = KNeighborsClassifier(n_neighbors=3)
        clf.fit(X_train, y_train) 
        y_pred = clf.predict(X_test)
        acc = accuracy_score(y_test, y_pred)
        resolutions_accs[resolution_idx].append(acc)

        

In [15]:
# Report the results
resolution_df = pd.DataFrame(resolutions_accs, columns=['person_1_accuracy', 'person_2_accuracy', 'person_3_accuracy'])
resolution_df.index = ['0.25','0.5','1']

m = resolution_df.mean(axis=1)
s = resolution_df.std(axis=1)
resolution_df['mean_accuracy'] = m
resolution_df['std_accuracy'] = s

resolution_df.index.name = 'welch segment length[s]'
resolution_df = resolution_df.round(2)
resolution_df.to_excel("Resolution_selection_report.xlsx")
resolution_df

,person_1_accuracy,person_2_accuracy,person_3_accuracy,mean_accuracy,std_accuracy
welch segment length[s],,,,,
0.25,0.61,0.42,0.52,0.52,0.10
0.5,0.62,0.38,0.52,0.50,0.12
1,0.68,0.35,0.52,0.52,0.16
